In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

In [ ]:
random.seed = 20

## Load model

In [ ]:
from GNNPGRegressor import PGSAGE, PGANN, PGGCN
from MyDataset import MyDataset

## Set parameters

In [ ]:
# Set hyper-parameters
input_dim1 = 32
input_dim2 = 44
hidden_dim1 = 32
hidden_dim2 = 32
output_dim1 = 32
output_dim2 = 12

# Set device
device = 'gpu' if torch.cuda.is_available() else 'cpu'

## SAGE

In [ ]:
root = '../../train_val_test_dataset/IEEE_Case118/PG-SAGE'

# Load all data with empty input
dataset = MyDataset(root=root, data_list=[])

# Get train dataset
train_size = 700

train_dataset = dataset[:train_size]

# Create train loader
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Initialize the model
model = PGSAGE(input_dim1, input_dim2, hidden_dim1, hidden_dim2, output_dim1, output_dim2)
model.to(device)

#### SAGE model training

In [ ]:
# Set number of epochs
epochs = 500

# Set optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Set model to training mode
model.train()

# Model training and validation
epoch_loss = []
start_time = time.time()
for epoch in range(epochs):
    batch_loss = []
    for loader in train_loader:
        loader.to(device)

        optimizer.zero_grad()

        y_pred = model(loader.x, loader.edge_index)

        loss = F.mse_loss(y_pred, loader.y)

        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
    epoch_loss.append(np.mean(batch_loss))

end_time = time.time()
training_time = (end_time - start_time)/60

plt.figure(figsize=(5, 3))
plt.plot(epoch_loss)
plt.yscale('log')
plt.xlabel('Epoch')
plt.ylabel('MSE loss')
plt.title(f'Training time: {training_time:.2f}')
plt.show()
plt.close('all')

#### Save SAGE model

In [ ]:
torch.save(model, './trained_model/PGSAGE_model.pt')

#### SAGE model testing

In [ ]:
model_path = './trained_model/PGSAGE_model.pt'
model = torch.load(model_path)

model.eval()

# Testing dataset 
# Get test dataset
test_size = 200
test_dataset = dataset[-test_size:]
############ Shuffle cannot be true for testing data ##############
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
with torch.no_grad():
    for i, loader in enumerate(test_loader):
        # Get ground true
        ground_true = loader.y.detach().numpy()
        # Save ground true
        file_path = f'./model_evaluation/SAGE/PG_true/true_{i+1}.csv'
        pd.DataFrame(ground_true).to_csv(file_path, columns=None, index=None, header=None)

        # Get prediction
        pred = model(loader.x, loader.edge_index)
        pred = pred.detach().numpy()

        ############ Add postprocessing ##############
        pred[pred<30] = 0

        # Save prediction
        file_path = f'./model_evaluation/SAGE/PG_pred/pred_{i+1}.csv'
        pd.DataFrame(pred).to_csv(file_path, columns=None, index=None, header=None)


# Complete dataset
test_dataset = dataset[:]
############ Shuffle cannot be true for testing data ##############
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

with torch.no_grad():
    for i, loader in enumerate(test_loader):
        # Get ground true
        ground_true = loader.y.detach().numpy()
        # Save ground true
        file_path = f'./model_evaluation/SAGE/PG_true_all/true_{i+1}.csv'
        pd.DataFrame(ground_true).to_csv(file_path, columns=None, index=None, header=None)

        # Get prediction
        pred = model(loader.x, loader.edge_index)
        pred = pred.detach().numpy()

        ############ Add postprocessing ##############
        pred[pred<30] = 0

        # Save prediction
        file_path = f'./model_evaluation/SAGE/PG_pred_all/pred_{i+1}.csv'
        pd.DataFrame(pred).to_csv(file_path, columns=None, index=None, header=None)

## ANN

In [ ]:
root = '../../train_val_test_dataset/IEEE_Case118/PG-ANN'

# Load all data with empty input
dataset = MyDataset(root=root, data_list=[])

# Get train dataset
train_size = 700

train_dataset = dataset[:train_size]

# Create train loader
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Initialize the model
model = PGANN(input_dim1, input_dim2, hidden_dim1, hidden_dim2, output_dim1, output_dim2)
model.to(device)

#### ANN model training

In [ ]:
# Set number of epochs
epochs = 500

# Set optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Set model to training mode
model.train()

# Model training and validation
epoch_loss = []
start_time = time.time()
for epoch in range(epochs):
    batch_loss = []
    for loader in train_loader:
        loader.to(device)

        optimizer.zero_grad()

        y_pred = model(loader.x, loader.edge_index)

        loss = F.mse_loss(y_pred, loader.y)

        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
    epoch_loss.append(np.mean(batch_loss))

end_time = time.time()
training_time = (end_time - start_time)/60

plt.figure(figsize=(5, 3))
plt.plot(epoch_loss)
plt.yscale('log')
plt.xlabel('Epoch')
plt.ylabel('MSE loss')
plt.title(f'Training time: {training_time:.2f}')
plt.show()
plt.close('all')

#### Save ANN model

In [ ]:
torch.save(model, './trained_model/PGANN_model.pt')

#### ANN model testing

In [ ]:
model_path = './trained_model/PGANN_model.pt'
model = torch.load(model_path)

model.eval()

# Testing dataset 
# Get test dataset
test_size = 200
test_dataset = dataset[-test_size:]
############ Shuffle cannot be true for testing data ##############
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
with torch.no_grad():
    for i, loader in enumerate(test_loader):
        # Get ground true
        ground_true = loader.y.detach().numpy()
        # Save ground true
        file_path = f'./model_evaluation/ANN/PG_true/true_{i+1}.csv'
        pd.DataFrame(ground_true).to_csv(file_path, columns=None, index=None, header=None)

        # Get prediction
        pred = model(loader.x, loader.edge_index)
        pred = pred.detach().numpy()

        ############ Add postprocessing ##############
        pred[pred<30] = 0

        # Save prediction
        file_path = f'./model_evaluation/ANN/PG_pred/pred_{i+1}.csv'
        pd.DataFrame(pred).to_csv(file_path, columns=None, index=None, header=None)


# Complete dataset
test_dataset = dataset[:]
############ Shuffle cannot be true for testing data ##############
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
with torch.no_grad():
    for i, loader in enumerate(test_loader):
        # Get ground true
        ground_true = loader.y.detach().numpy()
        # Save ground true
        file_path = f'./model_evaluation/ANN/PG_true_all/true_{i+1}.csv'
        pd.DataFrame(ground_true).to_csv(file_path, columns=None, index=None, header=None)

        # Get prediction
        pred = model(loader.x, loader.edge_index)
        pred = pred.detach().numpy()

        ############ Add postprocessing ##############
        pred[pred<30] = 0

        # Save prediction
        file_path = f'./model_evaluation/ANN/PG_pred_all/pred_{i+1}.csv'
        pd.DataFrame(pred).to_csv(file_path, columns=None, index=None, header=None)

## GCN

In [ ]:
root = '../../train_val_test_dataset/IEEE_Case118/PG-GCN'

# Load all data with empty input
dataset = MyDataset(root=root, data_list=[])

# Get train dataset
train_size = 700

train_dataset = dataset[:train_size]

# Create train loader
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Initialize the model
model = PGGCN(input_dim1, input_dim2, hidden_dim1, hidden_dim2, output_dim1, output_dim2)
model.to(device)

#### GCN model training

In [ ]:
# Set number of epochs
epochs = 500

# Set optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Set model to training mode
model.train()

# Model training and validation
epoch_loss = []
start_time = time.time()
for epoch in range(epochs):
    batch_loss = []
    for loader in train_loader:
        loader.to(device)

        optimizer.zero_grad()

        y_pred = model(loader.x, loader.edge_index)

        loss = F.mse_loss(y_pred, loader.y)

        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
    epoch_loss.append(np.mean(batch_loss))

end_time = time.time()
training_time = (end_time - start_time)/60

plt.figure(figsize=(5, 3))
plt.plot(epoch_loss)
plt.yscale('log')
plt.xlabel('Epoch')
plt.ylabel('MSE loss')
plt.title(f'Training time: {training_time:.2f}')
plt.show()
plt.close('all')

#### Save GCN model

In [ ]:
torch.save(model, './trained_model/PGGCN_model.pt')

#### GCN model testing

In [ ]:
model_path = './trained_model/PGGCN_model.pt'
model = torch.load(model_path)

model.eval()

# Testing dataset 
# Get test dataset
test_size = 200
test_dataset = dataset[-test_size:]
############ Shuffle cannot be true for testing data ##############
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
with torch.no_grad():
    for i, loader in enumerate(test_loader):
        # Get ground true
        ground_true = loader.y.detach().numpy()
        # Save ground true
        file_path = f'./model_evaluation/GCN/PG_true/true_{i+1}.csv'
        pd.DataFrame(ground_true).to_csv(file_path, columns=None, index=None, header=None)

        # Get prediction
        pred = model(loader.x, loader.edge_index)
        pred = pred.detach().numpy()

        ############ Add postprocessing ##############
        pred[pred<30] = 0

        # Save prediction
        file_path = f'./model_evaluation/GCN/PG_pred/pred_{i+1}.csv'
        pd.DataFrame(pred).to_csv(file_path, columns=None, index=None, header=None)


# Complete dataset
test_dataset = dataset[:]
############ Shuffle cannot be true for testing data ##############
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
with torch.no_grad():
    for i, loader in enumerate(test_loader):
        # Get ground true
        ground_true = loader.y.detach().numpy()
        # Save ground true
        file_path = f'./model_evaluation/GCN/PG_true_all/true_{i+1}.csv'
        pd.DataFrame(ground_true).to_csv(file_path, columns=None, index=None, header=None)

        # Get prediction
        pred = model(loader.x, loader.edge_index)
        pred = pred.detach().numpy()

        ############ Add postprocessing ##############
        pred[pred<30] = 0

        # Save prediction
        file_path = f'./model_evaluation/GCN/PG_pred_all/pred_{i+1}.csv'
        pd.DataFrame(pred).to_csv(file_path, columns=None, index=None, header=None)